In [1]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [2]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [3]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [4]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [5]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [6]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:05,  1.21it/s]


In [7]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.641714,3.641714,0.000000,9.641714,-4.358286,-0.358286,0.0,7.641714,...,9.641714,-4.358286,-0.358286,0.0,7.641714,3.641714,0.000000,9.641714,-4.358286,-0.358286
1,2025-01-22,0.0,7.641714,3.641714,0.000000,9.641714,-4.358286,-0.358286,0.0,7.641714,...,9.641714,-4.358286,-0.358286,0.0,7.641714,3.641714,0.000000,9.641714,-4.358286,-0.358286
2,2025-01-23,0.0,3.005869,3.641714,-13.278647,3.666099,-4.358286,-0.358286,0.0,-3.372618,...,-4.555784,-4.358286,-0.358286,0.0,9.384355,3.641714,-13.278647,11.887983,-4.358286,-0.358286
3,2025-01-24,0.0,3.005869,3.641714,-13.278647,3.666099,-4.358286,-0.358286,0.0,-3.372618,...,-4.555784,-4.358286,-0.358286,0.0,9.384355,3.641714,-13.278647,11.887983,-4.358286,-0.358286
4,2025-01-25,0.0,3.005869,3.641714,-13.278647,3.666099,-4.358286,-0.358286,0.0,-3.372618,...,-4.555784,-4.358286,-0.358286,0.0,9.384355,3.641714,-13.278647,11.887983,-4.358286,-0.358286


In [8]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [9]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [10]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://angusreid.org/one-year-of-don...",2026-01-16,2026-01-20,1838 A,1.229040e-01,37.0,56.0,-19.0,-17.157062
1,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,9.205869e-02,40.0,56.0,-16.0,-15.465108
2,"<a href=""https://emersoncollegepolling.com/jan...",2026-01-17,2026-01-19,1000 LV,8.600414e-02,43.2,51.3,-8.1,-14.105777
3,"<a href=""https://napolitannews.org/jobapproval...",2026-01-07,2026-01-14,3000 RV,6.583409e-02,48.0,51.0,-3.0,-12.953515
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-18,1095 A,6.402764e-02,39.0,57.0,-18.0,-16.220964
...,...,...,...,...,...,...,...,...,...
535,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,7.919226e-20,52.0,44.0,8.0,3.711993
536,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,7.526669e-20,49.0,36.0,13.0,8.159609
537,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,7.056264e-20,51.9,45.3,6.6,-0.002547
538,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,6.461161e-20,47.0,41.0,6.0,10.004615


In [11]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [12]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
52,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-16,2026-01-18,Trade/tariffs,2201 RV,1.214784e-02,45.0,46.0,-1.0,-13.605714
53,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,Trade/tariffs,1602 A,7.869424e-03,37.0,56.0,-19.0,-20.491595
54,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-09,2026-01-12,Trade/tariffs,2201 RV,7.664769e-03,43.0,48.0,-5.0,-17.605714
55,"<a href=""https://s3.documentcloud.org/document...",2026-01-09,2026-01-12,Trade/tariffs,1209 A,7.040978e-03,37.0,62.0,-25.0,-20.016743
56,"<a href=""https://www.gelliottmorris.com/p/new-...",2026-01-14,2026-01-20,Trade/tariffs,1532 A,4.157262e-03,38.0,57.0,-19.0,-19.064210
...,...,...,...,...,...,...,...,...,...,...
1254,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,3.810565e-14,47.0,37.0,10.0,8.508405
1255,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,2.198438e-14,48.0,37.0,11.0,9.508405
1256,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,1.271053e-14,45.0,41.0,4.0,2.508405
1257,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,7.448212e-15,48.0,35.0,13.0,11.508405


In [13]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [14]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,1.565050e-01,48.0,43.0,-5.0,-5.457773
1,"<a href=""https://emersoncollegepolling.com/jan...",2026-01-17,2026-01-19,1000 LV,1.462119e-01,48.1,41.7,-6.4,-7.685138
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-19,1549 RV,8.340804e-02,43.0,39.0,-4.0,-4.177739
3,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-12,2026-01-18,12505 RV,6.326562e-02,44.0,42.0,-2.0,-4.310389
4,"<a href=""https://www.yahoo.com/news/us/article...",2026-01-08,2026-01-12,1148 RV,5.772719e-02,45.0,40.0,-5.0,-5.177739
...,...,...,...,...,...,...,...,...,...
269,"<a href=""https://www.cygn.al/voters-back-kingm...",2025-01-09,2025-01-12,1500 LV,9.681050e-20,44.6,46.8,2.2,1.233316
270,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,8.647333e-20,40.0,48.0,8.0,2.996269
271,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,5.285971e-20,45.0,48.0,3.0,1.147288
272,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,2.799828e-21,44.6,48.4,3.8,2.833316


In [15]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)